In [1]:
import json
import pathlib
import os
import pickle
import numpy as np
from tqdm import tqdm
import copy
import glob
from gfootball.env.wrappers import Simple115StateWrapper

In [2]:
episodes_path = [path for path in glob.glob('episodes/*.json') if not '_info' in path]

In [3]:
def convert_observation(observation):
    return Simple115StateWrapper.convert_observation(observation['players_raw'], True)

In [4]:
for episode_path in tqdm(episodes_path):
    actions = []
    states = []
    rewards = []
    episode_id = pathlib.Path(episode_path).stem
    pickle_path = 'converted_episodes/{}.pickle'.format(episode_id)
    if not os.path.isfile(pickle_path):
        with open(episode_path, 'rt') as f:
            episode_dict = json.load(f)
        try:
            winner_index = np.argmax(episode_dict['rewards'])
            for step in episode_dict['steps']:
                actions.append(step[winner_index]['action'])
                states.append(convert_observation(step[winner_index]['observation']))
                rewards.append(step[winner_index]['reward'])
            episode = {
                "action": actions,
                "state": states,
                "reward": rewards
            }
            with open(pickle_path, 'wb') as f:
                pickle.dump(episode, f)
        except:
            pass

100%|██████████| 2308/2308 [00:00<00:00, 2684.82it/s]


## Data Augumentation

サッカーは上下対称なので、上下が反転した観測も水増しできる。

In [5]:
def reversed_observation(observation):
    reversed_observation = copy.deepcopy(observation[0])
    reversed_observation[1:22*4:2] *= -1
    reversed_observation[22*4+1:22*4+3*2+1:3] *= -1
    return reversed_observation[np.newaxis,:]

In [6]:
_reversed_actions_dict = {i: i for i in range(19)}
_reversed_actions_dict[2] = 8
_reversed_actions_dict[8] = 2
_reversed_actions_dict[3] = 7
_reversed_actions_dict[7] = 3
_reversed_actions_dict[4] = 6
_reversed_actions_dict[6] = 4

def reversed_action(action):
    if action == []:
        return []
    else:
        return [_reversed_actions_dict[action[0]]]

In [7]:
episodes_path = [path for path in glob.glob('./converted_episodes/*.pickle') if not 'r' in pathlib.Path(path).stem]

In [8]:
for path in tqdm(episodes_path):
    stem = pathlib.Path(path).stem
    path_reversed = path.replace('.pickle', 'r.pickle')
    if not pathlib.Path(path_reversed).is_file():
        with open(path, 'rb') as f:
            episode = pickle.load(f)
        reversed_episode = {
            "action": [reversed_action(action) for action in actions],
            "state": [reversed_observation(state) for state in states],
            "reward": rewards
        }
        with open(path_reversed, 'wb') as f:
             pickle.dump(reversed_episode, f)

100%|██████████| 2300/2300 [00:01<00:00, 1227.97it/s]


## Copy files into dataset dir

In [10]:
!cp converted_episodes/*.pickle dataset/raw/